In [ ]:
%load_ext autoreload
%autoreload 2

from functools import partial
from copy import deepcopy

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
from tqdm import autonotebook as tqdm

from ex1.main import Simulation
from ex1.main import read_scenario

rc('animation', html='jshtml')
%matplotlib inline

In [ ]:
scenario = read_scenario('scenario_task_3.json')
states = Simulation(scenario).get_states()

In [ ]:
fig, ax = plt.subplots()
ax.set_xticks(np.arange(0, scenario.field[1] + 1, 1))
ax.set_yticks(np.arange(0, scenario.field[0] + 1, 1))
plt.grid()

ax.set_xlim((0, scenario.field[1]))
ax.set_ylim((0, scenario.field[0]))

# line, = ax.plot([], [], lw=2)
colors = ['b', 'r', 'g']
cmap = matplotlib.colors.ListedColormap(colors)
sc_plot = ax.scatter([], [], c=[], cmap=cmap)

In [ ]:
offsets = []
colors = []
for timestamp, state in states:
    points = np.asarray((list(state.pedestrians) + list(state.obstacles) + list(state.target)))
    points[..., 0], points[..., 1] = points[..., 1], points[..., 0].copy()
    offsets.append(points)
    color = np.concatenate((
        np.zeros(len(state.pedestrians)),
        np.zeros(len(state.obstacles)) + 1,
        np.zeros(len(state.target)) + 2,
    ))
    colors.append(color)


def init():
    sc_plot.set_offsets([[0, 0]])
    return (sc_plot,)


pbar = tqdm.tqdm(total=len(states))


def animate(i, sc_plot):
    pbar.update(1)
    sc_plot.set_offsets(offsets[i])
    sc_plot.set_array(colors[i])
    return (sc_plot,)


anim = animation.FuncAnimation(fig, partial(animate, sc_plot=sc_plot), init_func=init,
                               frames=len(states), interval=500, blit=True)
anim

In [ ]:
#HTML(anim.to_html5_video())